# Conversion of the Latest Dataframe to Parquet

We need to store our dataset in a warehouse so we use parquet

In [11]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Load the Pandas DataFrame
df = pd.read_csv('dialogues.csv', sep='\t', encoding='utf-8')

In [12]:
# Convert Pandas DataFrame to Arrow Table
table = pa.Table.from_pandas(df)
# Specify the output file path for the Parquet file
parquet_file_path = './data/parquet/dialogues.parquet'

# Write the Arrow Table to a Parquet file
pq.write_table(table, parquet_file_path)

print(f'DataFrame saved to Parquet file: {parquet_file_path}')

DataFrame saved to Parquet file: ./data/parquet/dialogues.parquet
